<a href="https://colab.research.google.com/github/satani99/langchain_pdf/blob/main/Langchain_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain

# Install vectorStore
!pip install faiss-cpu

# Install gpt4all
!pip install gpt4all

# Install huggingfaceHub
!pip install huggingface-hub

# Install PyPdf for working with PDFs
!pip install pypdf

#Install Chroma
!pip install chromadb

#Install sentence-transformers
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.1

In [2]:
from langchain.document_loaders import PyPDFLoader
# from langchain import PromptTemplate, LLMChain
from langchain.embeddings import SentenceTransformerEmbeddings
# from langchain.llms import GPT4All
from langchain.text_splitter import CharacterTextSplitter
# from langchain.vectorstores.faiss import FAISS
from langchain.vectorstores import Chroma
# from langchain.callbacks.base import BaseCallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline

Upload all the PDFs in folder name 'docs'

In [3]:
loader = DirectoryLoader('docs', show_progress=True, loader_cls=PyPDFLoader)
docs = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
chroma_index = Chroma.from_documents(texts, embeddings, persist_directory="index")
chroma_index.persist()

100%|██████████| 6/6 [00:08<00:00,  1.49s/it]


In [4]:
print("loading indexes")
chroma_index = Chroma(persist_directory="/content/index", embedding_function=embeddings)
print("index loaded")

loading indexes
index loaded


In [5]:
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    model_kwargs={"temperature": 0.2, "max_length": 1024},
    device=0
)

In [6]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=chroma_index.as_retriever())

In [ ]:
while True:
  query = input()
  print(qa.run(query))